# 向量数据库的使用
+ 向量库的数据增加
+ 向量库的数据删除
+ 向量库的相似性搜索
+ 高级使用：MMMR
+ 高级使用：混合搜索

In [12]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [3]:
from langchain_openai import OpenAIEmbeddings
import os

embeddings_model = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    # https://api.siliconflow.cn/v1/embeddings
    base_url=os.environ.get("SILICONFLOW_API_BASE"),
    api_key=os.environ.get("SILICONFLOW_API_KEY"),
)

## 向量库的数据增加

为了演示方便引入内存向量数据库，它将向量暂存在内存中，并使用字典以及numpy计算搜索的余玄相似度

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding=embeddings_model)

In [6]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="今天在抖音学会了一个新菜:锅巴土豆泥!看起来简单，实际炸了厨房，连猫都嫌弃地走开了。",
    metadata={"source": "社交媒体"},
)

document_2 = Document(
    page_content="小区遛狗大爷今日播报:广场舞大妈占领健身区，遛狗群众纷纷撤退。现场气氛诡异，BGM已循环播放《最炫民族风》两小时。",
    metadata={"source": "社区新闻"},
)

documents = [document_1, document_2]

# 添加到向量数据库中
vector_store.add_documents(documents=documents)

['ac2b6080-0a83-4b4f-820f-f2b98dcbd326',
 '5f612340-0b84-41fe-be06-d0809d71c0ce']

In [7]:
# 可以为添加的文档增加ID索引，便于后面管理
vector_store.add_documents(documents=documents, ids=["doc1", "doc2"])

['doc1', 'doc2']

## 向量库的数据删除

In [8]:
vector_store.delete(id=["doc1"])

## 向量库的相似性搜索

In [ ]:
# 关键词搜索
query = "遛狗"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

小区遛狗大爷今日播报:广场舞大妈占领健身区，遛狗群众纷纷撤退。现场气氛诡异，BGM已循环播放《最炫民族风》两小时。


In [10]:
# 使用向量查相似向量的方式来搜索

embedding_vector = embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

小区遛狗大爷今日播报:广场舞大妈占领健身区，遛狗群众纷纷撤退。现场气氛诡异，BGM已循环播放《最炫民族风》两小时。


注意：langchain只是在接口层面进行了封装，具体的搜索实现要依赖向量库本身的能力，比如Pinecone就可以进行元数据过滤，内存向量就不可以

需要去Pinecone官网申请账号获取API
https://docs.pinecone.io/guides/get-started/quickstart

### 使用Pinecone向量数据库进行元数据过滤

In [11]:
! pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter you Pinecone API Key:")

pinecone_api_key = os.getenv("PINECONE_API_KEY")

# 创建Pinecone客户端
pc = Pinecone(api_key=pinecone_api_key)

In [22]:
# 初始化向量数据库
import time

index_name = "langchain-pinecone-test"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    # 如果索引不存在，创建它
    pc.create_index(
        name=index_name,
        dimension=1024,  # 注意维度要和嵌入模型维度一致
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )
    # 等待索引初始化
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [24]:
# 添加嵌入模型
from langchain_pinecone.vectorstores import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [30]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="今天早餐吃了老王家的生煎包，馅料实在得快从褶子里跳出来了!这才是真正的上海味道!",
    metadata={"source": "tweet"},
)
document_2 = Document(
    page_content="明日天气预报:北京地区将出现大范围雾霾，建议市民戴好口罩，看不见脸的时候请不要慌张。",
    metadata={"source": "news"},
)
document_3 = Document(
    page_content="终于搞定了AI聊天机器人!我问它'你是谁'，它回答'我是你爸爸'，看来还需要调教...",
    metadata={"source": "tweet"},
)
document_4 = Document(
    page_content="震惊!本市一男子在便利店抢劫，只因店员说'扫码支付才有优惠'，现已被警方抓获。",
    metadata={"source": "news"},
)
document_5 = Document(
    page_content="刚看完《流浪地球3》,特效简直炸裂!就是旁边大妈一直问'这是在哪拍的'有点影响观影体验。",
    metadata={"'source": "tweet"},
)
document_6 = Document(
    page_content="新发布的小米14Ultra值不值得买?看完这篇测评你就知道为什么李老板笑得合不拢嘴了。",
    metadata={"source": "website"},
)
document_7 = Document(
    page_content="2025年中超联赛十大最佳球员榜单新鲜出炉，第一名居然是他?!",
    metadata={"source": "website"},
)
document_8 = Document(
    page_content="用LangChain开发的AI助手太神奇了！问它'人生的意义'，它给我推荐了一份外卖优惠券...",
    metadata={"source": "tweet"},
)
document_9 = Document(
    page_content="A股今日暴跌，分析师称原因是'大家都在抢着卖'，投资者表示很有道理。",
    metadata={"source": "news"},
)
document_10 = Document(
    page_content="感觉我马上要被删库跑路了，祝我好运 /(ToT)/~~",
    metadata={"source": "tweet"},
)


documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# 为文档生成ID
uuids = [str(uuid4()) for _ in range(len(documents))]

# 将文档和生成的ID存储到向量数据库中
vector_store.add_documents(documents=documents, ids=uuids)

['fb766ff3-0ea2-4f81-b50a-c88f8ddd175b',
 'd291d4d1-6e21-4c10-b557-df9617767a84',
 '913b8eaf-a4e2-41a8-8605-be5fb62cca19',
 '1caecebb-a84f-426e-b0de-65928a03551e',
 'b05d9d86-3e7a-4d3b-93c1-9f0e26507f3e',
 '7516c346-7732-44bd-802f-20881e825d05',
 'de369a04-5be5-4f9b-af0b-5c7de688d18c',
 'a98b91ef-f9c4-47c3-adca-eb42586e2d17',
 'eb9b4812-81ae-499c-959d-10f8c19b8960',
 '74cd3909-2e42-44f5-90b1-1f53a63f053c']

In [31]:
# 删除向量数据库中的数据

vector_store.delete(ids=[uuids[-1]])

In [32]:
# 相似性搜索支持元数据过滤

results = vector_store.similarity_search(
    "看电影",
    k=2,
    filter={"source": "tweet"},
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 用LangChain开发的AI助手太神奇了！问它'人生的意义'，它给我推荐了一份外卖优惠券... [{'source': 'tweet'}]
* 今天早餐吃了老王家的生煎包，馅料实在得快从褶子里跳出来了!这才是真正的上海味道! [{'source': 'tweet'}]


In [34]:
# 通过分输进行过滤
results = vector_store.similarity_search_with_score(
    "明天热吗？",
    k=2,
    filter={"source": "news"},
)

for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.630674] 明日天气预报:北京地区将出现大范围雾霾，建议市民戴好口罩，看不见脸的时候请不要慌张。 [{'source': 'news'}]
* [SIM=0.558014] A股今日暴跌，分析师称原因是'大家都在抢着卖'，投资者表示很有道理。 [{'source': 'news'}]


## MMR（最大边际相关性）
+ 并非所有向量数据库都支持

In [37]:
vector_store.max_marginal_relevance_search(
    query="新手机",
    k=3,
    lambda_val=0.1,
    filter={"source": "website"},
)

[Document(metadata={'source': 'website'}, page_content='2025年中超联赛十大最佳球员榜单新鲜出炉，第一名居然是他?!'),
 Document(metadata={'source': 'website'}, page_content='新发布的小米14Ultra值不值得买?看完这篇测评你就知道为什么李老板笑得合不拢嘴了。')]

In [ ]:
## 混合搜索

In [38]:
! pip install pinecone-text pinecone-notebooks

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pinecone-text]


In [39]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

# 初始化Pinecone索引

index_name = "langchain-pinecone-hybrid-search-test"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    # 如果索引不存在，创建它
    pc.create_index(
        name=index_name,
        dimension=1024,  # 注意维度要和嵌入模型维度一致
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )
    # 等待索引初始化
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [40]:
# 先使用词法过滤（关键词匹配）
from pinecone_text.sparse import BM25Encoder

# 初始化词法引擎
bm25_encoder = BM25Encoder().default()

[nltk_data] Downloading package stopwords to /home/jizhe/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [41]:
corpus = ["foo", "bar", "world", "hello"]

bm25_encoder.fit(corpus)

bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 4/4 [00:00<00:00, 121.86it/s]


In [ ]:
# 使用混合搜索
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings_model,
    sparse_encoder=bm25_encoder,  # 传入词法引擎
    index=index,
    top_k=5,
)

In [43]:
retriever.add_texts(["foo", "bar", "world", "hello"])

100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [44]:
result = retriever.invoke("foo")
result[0]

Document(metadata={'score': 0.484042168}, page_content='hello')